# =============================================================================
# DENDRIC CLUSTERING ANALYSIS - MAIN NOTEBOOK
# =============================================================================
# This notebook performs comprehensive dendritic clustering analysis of neuronal
# synapse data, including excitatory/inhibitory synapse mapping, clustering,
# correlation analysis, and visualization.


In [1]:
# Find the project root (the folder that contains 'src' and 'config') and add it to sys.path
import sys
from pathlib import Path

def add_project_root(markers=("src", "config")):
    here = Path.cwd()
    for p in (here, *here.parents):
        if all((p / m).exists() for m in markers):
            sys.path.insert(0, str(p))
            print(" Project root added to sys.path:", p)
            return p
    raise RuntimeError("Could not locate project root with markers:", markers)

PROJECT_ROOT = add_project_root()


 Project root added to sys.path: /home/ge95zic/mnt/ge95zic/dendric_clustering/refactored


In [2]:
# =============================================================================
# 1. CONFIGURATION AND SETUP
# =============================================================================
# Load configuration from YAML file and set up project paths and parameters

from src.config import load_config
from src.utils import ensure_dir, set_seed
import importlib, src.config as cfgmod
importlib.reload(cfgmod)

from src.config import load_config, Paths
print("Paths dataclass fields:", Paths.__annotations__)  # should include 'interim_results_dir'
cfg = load_config("../config/default.yaml")
print(cfg.paths)


Paths dataclass fields: {'base_data': <class 'str'>, 'swc_dir': <class 'str'>, 'syn_dir': <class 'str'>, 'interim_results_dir': <class 'str'>}
Paths(base_data='data', swc_dir='data/microns/extracted_swc/SWC_Skel', syn_dir='data/microns/extracted_swc/syn', interim_results_dir='data/microns/intirim_results')


In [3]:
# =============================================================================
# 2. NEURON SKELETON LOADING
# =============================================================================
# Load and heal the SWC neuron skeleton file

from pathlib import Path
from src.io_swc import load_and_heal_swc

swc_dir_abs = (PROJECT_ROOT / cfg.paths.swc_dir).resolve()
swc_path = swc_dir_abs / f"{cfg.input.swc_prefix}{cfg.input.neuron_id}.swc"

print("SWC path:", swc_path, "exists:", swc_path.exists())

neuron = load_and_heal_swc(swc_path)
print("Neuron loaded:", neuron)

neuron.nodes


SWC path: /home/ge95zic/mnt/ge95zic/dendric_clustering/refactored/data/microns/extracted_swc/SWC_Skel/microns_jan_n3.swc exists: True
Neuron loaded: type                                             navis.TreeNeuron
name                                               microns_jan_n3
id                           431c6796-76a6-46af-a686-6b24dc0c7b1c
n_nodes                                                     31521
n_connectors                                                 None
n_branches                                                    219
n_leafs                                                       230
cable_length                                          3213.320312
soma                                                         None
units                                                1 micrometer
created_at                             2025-09-12 16:56:22.319090
origin          /home/ge95zic/mnt/ge95zic/dendric_clustering/r...
file                                           microns_jan_

,node_id,label,x,y,z,radius,parent_id,type
0,1,0,651.012634,231.868530,906.797058,0.791176,-1,root
1,2,0,662.086609,220.090561,895.135742,1.009817,4,slab
2,3,0,652.394836,226.683167,906.061157,1.217976,29273,end
3,4,0,657.215271,213.973145,897.149963,3.298922,212,branch
4,5,0,563.103943,191.319458,847.732666,0.627453,1901,slab
...,...,...,...,...,...,...,...,...
31516,31517,0,703.483521,241.006805,859.725708,0.522779,31518,slab
31517,31518,0,703.466675,240.917740,859.672852,0.559932,31519,slab
31518,31519,0,703.449768,240.828690,859.619995,0.597086,31520,slab
31519,31520,0,703.432861,240.739624,859.567139,0.634239,31521,slab


In [4]:
# =============================================================================
# 4. SYNAPSE DATA LOADING AND PREPROCESSING
# =============================================================================
# Load synapse data, deduplicate, compute volumes, and split into E/I types

from pathlib import Path
from src.datasets.microns import build_syn_path, read_synapses_raw
from src.prep import (
    dedupe_raw_xyz,
    add_synapse_volume_nm3_to_um3,
    split_e_i_by_isOnSpine,
    normalized_from_raw,
    compute_volume_um3,
    drop_duplicate_positions,
)

# Build absolute path
syn_dir_abs = (PROJECT_ROOT / cfg.paths.syn_dir).resolve()
syn_path = build_syn_path(syn_dir_abs, cfg.input.syn_prefix, cfg.input.neuron_id)
print("Syn file:", syn_path)

# 1) Read raw synapse data
df_raw = read_synapses_raw(syn_path)

# 2) ORIGINAL PIPELINE (raw layer) - deduplicate and compute volumes
df_raw = dedupe_raw_xyz(df_raw, x="x", y="y", z="z")
df_raw = add_synapse_volume_nm3_to_um3(df_raw,
                                       syn_size_col="syn_size",
                                       out_col="SynapseVolume",
                                       dx_nm=cfg.voxel_nm.dx, dy_nm=cfg.voxel_nm.dy, dz_nm=cfg.voxel_nm.dz)
syn_exec_df, syn_inh_df = split_e_i_by_isOnSpine(df_raw)

# 3) NORMALIZED PIPELINE (for new functions) - standardize coordinate system
syn_df_norm = normalized_from_raw(df_raw, pos_unit="nm")
syn_df_norm = compute_volume_um3(syn_df_norm, dx_nm=cfg.voxel_nm.dx, dy_nm=cfg.voxel_nm.dy, dz_nm=cfg.voxel_nm.dz)
syn_df_norm = drop_duplicate_positions(syn_df_norm)

# Quick checks
print(f"All (raw, post-dedupe): {len(df_raw)} | E: {len(syn_exec_df)} | I: {len(syn_inh_df)}")
print("Normalized columns:", list(syn_df_norm.columns))
display(syn_exec_df.head())
display(syn_inh_df.head())


Syn file: /home/ge95zic/mnt/ge95zic/dendric_clustering/refactored/data/microns/extracted_swc/syn/microns_jan_synn3.xlsx
All (raw, post-dedupe): 1423 | E: 959 | I: 464
Normalized columns: ['x_um', 'y_um', 'z_um', 'syn_size', 'is_on_spine', 'type', 'volume_um3']


,Unnamed: 0,id,Epos3DX,Epos3DY,Epos3DZ,syn_size,pre_identity,pre_type,isOnSpine,SynapseVolume
1,1,30202,657.473904,178.620753,914.00,24780,864691135593788160,Unknown,0,0.099120
2,2,30501,700.190165,140.783739,915.72,8124,864691135994517376,Unknown,0,0.032496
7,7,19743,635.829279,185.255547,878.16,21756,864691135048467328,Unknown,0,0.087024
8,8,19350,664.358232,172.629965,890.64,6696,864691136585704960,Unknown,0,0.026784
12,12,24523,648.174247,161.826327,851.84,1132,864691135469724288,DTC,0,0.004528


,Unnamed: 0,id,Ipos3DX,Ipos3DY,Ipos3DZ,syn_size,pre_identity,pre_type,isOnSpine,SynapseVolume
0,0,6402,709.784696,228.465614,882.92,15004,864691135403564800,Unknown,1,0.060016
3,3,11853,621.166912,257.018718,897.28,4112,864691136585348608,Unknown,1,0.016448
4,4,23503,643.704676,157.564561,890.00,392,864691135437611008,Unknown,1,0.001568
5,5,19873,638.696607,157.254900,834.36,8300,864691135469724288,DTC,1,0.033200
6,6,17974,640.330171,33.759337,832.32,160,864691135453276288,Unknown,1,0.000640


In [5]:
# =============================================================================
# 5. SYNAPSE-TO-NODE MAPPING
# =============================================================================
# Map excitatory synapses to their closest skeleton nodes and compute statistics

import numpy as np
from src.processing.synapse_mapping import (
    map_synapses_to_nodes,
    compute_synapse_node_statistics,
    create_node_counts_series
)

# Map excitatory synapses to nodes
syn_exec_df = map_synapses_to_nodes(
    neuron, syn_exec_df, 
    coord_cols=("Epos3DX", "Epos3DY", "Epos3DZ"),
    method="brute_force"
)

# Compute mapping statistics
mapping_stats = compute_synapse_node_statistics(syn_exec_df)
print("Excitatory synapse mapping statistics:")
for key, value in mapping_stats.items():
    print(f"  {key}: {value}")

# Create node counts series aligned with neuron skeleton
node_counts = create_node_counts_series(syn_exec_df, neuron)

# Map inhibitory synapses to nodes (for later analysis)
syn_inh_df = map_synapses_to_nodes(
    neuron, syn_inh_df,
    coord_cols=("Ipos3DX", "Ipos3DY", "Ipos3DZ"),
    method="brute_force"
)

print(f"\nInhibitory synapses mapped to {len(syn_inh_df['closest_node_id'].unique())} unique nodes")


Excitatory synapse mapping statistics:
  n_unique_nodes_with_synapses: 907
  max_synapses_per_node: 4
  max_synapses_node_id: 1
  avg_synapses_per_node: 1.0573318632855568
  median_synapses_per_node: 1.0
  total_synapses: 959

Inhibitory synapses mapped to 452 unique nodes


In [6]:
# =============================================================================
# 6. GEODESIC DISTANCE MATRIX COMPUTATION
# =============================================================================
# Compute or load cached geodesic distance matrix between all skeleton nodes

import navis
import os, pickle
from src.geo import compute_or_load_geodesic

# Build cache directory
interim_dir = (PROJECT_ROOT / cfg.paths.interim_results_dir).resolve()
interim_dir.mkdir(parents=True, exist_ok=True)

# Compute or load geodesic matrix
geodesic_mat_full = compute_or_load_geodesic(
    neuron, interim_dir, cfg.input.neuron_id, force_recompute=False
)

print(f"Geodesic matrix shape: {geodesic_mat_full.shape}")
print(f"Matrix type: {type(geodesic_mat_full)}")

# Check for infinite values
arr = geodesic_mat_full.values if hasattr(geodesic_mat_full, "values") else geodesic_mat_full
if np.isinf(arr).any():
    print(" The geodesic matrix contains infinite values.")
else:
    print(" Geodesic matrix is valid (no infinite values)")

geodesic_mat_full


Found geodesic cache: /home/ge95zic/mnt/ge95zic/dendric_clustering/refactored/data/microns/intirim_results/geodesic_mat_n3.pkl → loading
 Geodesic matrix contains ∞ values.
Geodesic matrix shape: (31521, 31521)
Matrix type: <class 'pandas.core.frame.DataFrame'>
 The geodesic matrix contains infinite values.


,1,2,3,4,5,6,7,8,9,10,...,31512,31513,31514,31515,31516,31517,31518,31519,31520,31521
1,0.0,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2,inf,0.000000,inf,inf,inf,inf,inf,inf,inf,inf,...,68.591893,68.486954,68.382024,68.277097,68.172189,68.067249,67.962319,67.857393,67.752453,67.647544
3,inf,inf,0.0,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
4,inf,inf,inf,0.000000,148.487333,148.488960,129.505555,129.507832,129.506371,136.053309,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
5,inf,inf,inf,148.487333,0.000000,0.204895,277.992889,277.995165,277.993704,12.639064,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31517,inf,68.067249,inf,inf,inf,inf,inf,inf,inf,inf,...,0.524644,0.419705,0.314775,0.209848,0.104940,0.000000,0.104930,0.209856,0.314796,0.419705
31518,inf,67.962319,inf,inf,inf,inf,inf,inf,inf,inf,...,0.629574,0.524635,0.419705,0.314778,0.209869,0.104930,0.000000,0.104927,0.209866,0.314775
31519,inf,67.857393,inf,inf,inf,inf,inf,inf,inf,inf,...,0.734501,0.629561,0.524631,0.419705,0.314796,0.209856,0.104927,0.000000,0.104940,0.209848
31520,inf,67.752453,inf,inf,inf,inf,inf,inf,inf,inf,...,0.839440,0.734501,0.629571,0.524644,0.419736,0.314796,0.209866,0.104940,0.000000,0.104909


In [7]:
# =============================================================================
# 7. GAUSSIAN KERNEL WEIGHT MATRIX COMPUTATION
# =============================================================================
# Compute Gaussian kernel weight matrix for synapse density smoothing

import pickle
import os
from src.geo import compute_or_load_node_kernel

# Compute or load node kernel weight matrix
W_nodes, W_nodes_normalized = compute_or_load_node_kernel(
    geodesic_mat_full, interim_dir, cfg.input.neuron_id, 
    cfg.analysis.sigma_um, force_recompute=False
)

print(f"Weight matrix shape: {W_nodes.shape}")
print(f"Weight matrix type: {type(W_nodes)}")
print(f"Sigma parameter: {cfg.analysis.sigma_um} μm")

# Check matrix properties
if hasattr(W_nodes, 'values'):
    # If it's a pandas DataFrame, get the underlying numpy array
    W_values = W_nodes.values
else:
    # If it's already a numpy array
    W_values = W_nodes

print(f"Min weight: {W_values.min():.6f}")
print(f"Max weight: {W_values.max():.6f}")
print(f"Mean weight: {W_values.mean():.6f}")

print(" Gaussian kernel weight matrix computed successfully")


Found file: /home/ge95zic/mnt/ge95zic/dendric_clustering/refactored/data/microns/intirim_results/node_weight_mat_n3_sigma:1.0.pkl. Loading node weight matrix...
Weight matrix shape: (31521, 31521)
Weight matrix type: <class 'pandas.core.frame.DataFrame'>
Sigma parameter: 1.0 μm
Min weight: 0.000000
Max weight: 1.000000
Mean weight: 0.000825
 Gaussian kernel weight matrix computed successfully


In [8]:
# =============================================================================
# 8. SYNAPSE DENSITY COMPUTATION AND GRADIENT-ASCENT CLUSTERING
# =============================================================================
# Compute synapse densities and perform gradient-ascent clustering

from src.density import node_density_from_counts, attach_density_to_nodes
from src.clustering import (
    add_synapse_counts,
    get_local_maximum,
    group_by_peak,
    summarize_clusters,
    add_peak_node_to_calculation_nodes,
)

# 1) Compute synapse density for each node
# This calculates the density per node using the Gaussian kernel weight matrix
# W_nodes_normalized.dot(node_counts) gives the smoothed synapse density
node_density = node_density_from_counts(W_nodes_normalized, node_counts)

# Print density statistics matching the original dc_initial_algo.py format
print(f"Maximum density: {node_density.max():.3f}")
print(f"Average density: {node_density.mean():.3f}") 
print(f"Minimum density: {node_density.min():.3f}")
print(f"Computed synapse density for {len(node_density)} nodes")

# 2) Attach density to neuron nodes
# This creates calculation_nodes with synapse_density column, matching original code
calculation_nodes = attach_density_to_nodes(neuron, node_density)
print(f"Attached density to {len(calculation_nodes)} nodes")

# 3) Add synapse counts to calculation nodes
calculation_nodes = add_synapse_counts(calculation_nodes, node_counts)

# 4) Use only nodes with synapses (>0) for clustering
synapse_node_list = calculation_nodes.loc[calculation_nodes["synapse_count"] > 0, "node_id"].tolist()
print(f"Nodes with synapses: {len(synapse_node_list)}")

# 5) Perform gradient-ascent clustering to find local density maxima
peaks = get_local_maximum(neuron, calculation_nodes, synapse_node_list)
peaks = {int(k): int(v) for k, v in peaks.items()}

# 6) Group nodes by their peak
clusters_dict = group_by_peak(peaks)

# 6.5) Add peak_node column to calculation_nodes
calculation_nodes = add_peak_node_to_calculation_nodes(calculation_nodes, clusters_dict)
print(f"Added peak_node column to calculation_nodes. Columns: {list(calculation_nodes.columns)}")

# 7) Summarize clustering results
summary = summarize_clusters(clusters_dict)
print("\nClustering Results:")
print(f"  Number of clusters: {summary['n_clusters']}")
print(f"  Largest cluster peak: {summary['largest_peak']}")
print(f"  Largest cluster size: {summary['largest_count']} nodes")
print(f"  Total assigned nodes: {summary['total_assigned_nodes']}")

# 8) Sanity check
total_nodes = sum(len(nodes) for nodes in clusters_dict.values())
print(f"  Sanity check - total nodes: {total_nodes}")

print(" Gradient-ascent clustering completed successfully")


Maximum density: 2.560
Average density: 0.285
Minimum density: 0.000
Computed synapse density for 31521 nodes
Attached density to 31521 nodes
Nodes with synapses: 907


Nodes processed:   0%|          | 0/907 [00:00<?, ?it/s]

Added peak_node column to calculation_nodes. Columns: ['node_id', 'label', 'x', 'y', 'z', 'radius', 'parent_id', 'type', 'synapse_density', 'synapse_count', 'peak_node']

Clustering Results:
  Number of clusters: 586
  Largest cluster peak: 13795
  Largest cluster size: 7 nodes
  Total assigned nodes: 907
  Sanity check - total nodes: 907
 Gradient-ascent clustering completed successfully


In [9]:
# =============================================================================
# 9. CLUSTER PROCESSING AND METRICS COMPUTATION
# =============================================================================
# Build cluster DataFrame and compute cluster metrics

# Force reload of the module to get updated code
import importlib
import src.processing.cluster_processing as cluster_processing
importlib.reload(cluster_processing)

from src.processing.cluster_processing import (
    build_cluster_dataframe,
    compute_cluster_metrics,
    filter_clusters_by_density,
    print_cluster_statistics
)

# 1) Create cluster ID mappings
unique_peaks = sorted(clusters_dict.keys())
peak_to_cluster_id = {peak: idx for idx, peak in enumerate(unique_peaks, start=0)}
new_cluster_id_to_peak = {new_id: peak for peak, new_id in peak_to_cluster_id.items()}

# 2) Assign cluster IDs to synapses and nodes
syn_exec_df["cluster_id"] = syn_exec_df["closest_node_id"].map(peaks).map(peak_to_cluster_id)
calculation_nodes["cluster_id"] = calculation_nodes["node_id"].map(peaks).map(peak_to_cluster_id)

# 3) Build cluster DataFrame
cluster_df = build_cluster_dataframe(
    syn_exec_df, clusters_dict, peak_to_cluster_id, new_cluster_id_to_peak, syn_id_col="id"
)

# 4) Compute cluster metrics (cable length, density)
cluster_df = compute_cluster_metrics(
    cluster_df, syn_exec_df, geodesic_mat_full, neuron
)

# 5) Filter clusters by density threshold
overall_density = len(syn_exec_df) / neuron.cable_length
cluster_df = filter_clusters_by_density(cluster_df, overall_density, min_density_factor=1.0)

print(f"\nCluster Processing Results:")
print(f"  Total clusters found: {len(unique_peaks)}")
print(f"  Clusters after density filtering: {len(cluster_df)}")
print(f"  Overall synapse density: {overall_density:.5f} synapses/μm")

# Print comprehensive cluster statistics
print_cluster_statistics(cluster_df)

print(" Cluster processing completed successfully")


Filtered clusters: 586 -> 206
Density threshold: 0.29845

Cluster Processing Results:
  Total clusters found: 586
  Clusters after density filtering: 206
  Overall synapse density: 0.29845 synapses/μm

CLUSTER STATISTICS

 SYNAPSE COUNT:
  Max  # syn:   8
  Mean # syn:   2.70
  Median # syn: 2.0
  Min  # syn:   2
  Std  # syn:   1.04

 CLUSTER DENSITY (synapses/μm):
  Max  density:   26.534
  Mean density:   3.110
  Median density: 1.751
  Min  density:   0.687
  Std  density:   4.184

 MINIMAL CABLE LENGTH (μm):
  Max  length:   7.610
  Mean length:   1.672
  Median length: 1.424
  Min  length:   0.079
  Std  length:   1.279

 SUMMARY:
  Total clusters: 206
  Total synapses: 557
  Total cable length: 344.45 μm
  Overall density: 1.617 synapses/μm
 Cluster processing completed successfully


In [10]:
# =============================================================================
# 10. INHIBITORY SYNAPSE ANALYSIS
# =============================================================================
# Analyze inhibitory synapses and their relationship to excitatory clusters

# 1) Compute inhibitory synapse density and clustering
node_counts_inh = create_node_counts_series(syn_inh_df, neuron, 'closest_node_id')
node_density_inh = node_density_from_counts(W_nodes_normalized, node_counts_inh)
calculation_nodes_inh = attach_density_to_nodes(neuron, node_density_inh)
calculation_nodes_inh = add_synapse_counts(calculation_nodes_inh, node_counts_inh)

# 2) Perform gradient-ascent clustering for inhibitory synapses
synapse_node_list_inh = calculation_nodes_inh.loc[calculation_nodes_inh["synapse_count"] > 0, "node_id"].tolist()
peaks_inh = get_local_maximum(neuron, calculation_nodes_inh, synapse_node_list_inh)
peaks_inh = {int(k): int(v) for k, v in peaks_inh.items()}
clusters_dict_inh = group_by_peak(peaks_inh)

# 2.5) Add peak_node column to calculation_nodes_inh
calculation_nodes_inh = add_peak_node_to_calculation_nodes(calculation_nodes_inh, clusters_dict_inh)
print(f"Added peak_node column to calculation_nodes_inh. Columns: {list(calculation_nodes_inh.columns)}")

# 3) Create inhibitory cluster ID mappings (similar to excitatory)
unique_peaks_inh = sorted(clusters_dict_inh.keys())
peak_to_cluster_id_inh = {peak: idx for idx, peak in enumerate(unique_peaks_inh, start=0)}
new_cluster_id_to_peak_inh = {new_id: peak for peak, new_id in peak_to_cluster_id_inh.items()}

# 4) Assign inhibitory cluster IDs to synapses and nodes
syn_inh_df["cluster_id"] = syn_inh_df["closest_node_id"].map(peaks_inh).map(peak_to_cluster_id_inh)
calculation_nodes_inh["cluster_id"] = calculation_nodes_inh["node_id"].map(peaks_inh).map(peak_to_cluster_id_inh)

# 5) Build inhibitory cluster DataFrame
syn_inh_cluster_df = build_cluster_dataframe(
    syn_inh_df, clusters_dict_inh, peak_to_cluster_id_inh, new_cluster_id_to_peak_inh, syn_id_col="id"
)

# 6) Compute inhibitory cluster metrics (cable length, density)
syn_inh_cluster_df = compute_cluster_metrics(
    syn_inh_cluster_df, syn_inh_df, geodesic_mat_full, neuron
)

print(f"\nInhibitory Analysis Results:")
print(f"  Inhibitory clusters found: {len(clusters_dict_inh)}")
print(f"  Inhibitory synapses: {len(syn_inh_df)}")

print(" Inhibitory synapse analysis completed successfully")


Nodes processed:   0%|          | 0/452 [00:00<?, ?it/s]

Added peak_node column to calculation_nodes_inh. Columns: ['node_id', 'label', 'x', 'y', 'z', 'radius', 'parent_id', 'type', 'synapse_density', 'synapse_count', 'peak_node']

Inhibitory Analysis Results:
  Inhibitory clusters found: 328
  Inhibitory synapses: 464
 Inhibitory synapse analysis completed successfully


In [12]:
syn_inh_df

,Unnamed: 0,id,Ipos3DX,Ipos3DY,Ipos3DZ,syn_size,pre_identity,pre_type,isOnSpine,SynapseVolume,closest_node_id,distance_to_node,cluster_id,closest_e_syn_id,min_dist_e_syn_tot,cluster_id_exec
0,0,6402,709.784696,228.465614,882.92,15004,864691135403564800,Unknown,1,0.060016,6755,0.182822,63,6445.0,4.297547,-1
3,3,11853,621.166912,257.018718,897.28,4112,864691136585348608,Unknown,1,0.016448,12127,0.399673,126,11844.0,0.904383,-1
4,4,23503,643.704676,157.564561,890.00,392,864691135437611008,Unknown,1,0.001568,23627,0.012481,253,19131.0,1.395142,362
5,5,19873,638.696607,157.254900,834.36,8300,864691135469724288,DTC,1,0.033200,20052,0.315540,216,19883.0,1.002282,-1
6,6,17974,640.330171,33.759337,832.32,160,864691135453276288,Unknown,1,0.000640,18179,0.383540,194,18007.0,3.306412,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1416,1416,19869,638.520280,157.159168,834.00,216,864691135082840576,PTC,1,0.000864,20048,0.221937,216,19883.0,1.403657,-1
1417,1417,23831,645.271543,250.904308,979.40,1972,864691135702215296,DTC,1,0.007888,23951,0.309359,257,23806.0,2.520981,-1
1418,1418,17240,662.866271,220.249137,894.24,3000,864691136389008640,L4b,1,0.012000,17453,0.269309,186,19405.0,14.422655,369
1420,1420,15762,632.046189,243.274609,897.56,1412,864691134919322112,DTC,1,0.005648,15997,0.433969,166,15768.0,0.601803,308


In [16]:
# =============================================================================
# 11. E/I DISTANCE-BASED MAPPING
# =============================================================================
# Map inhibitory synapses to excitatory clusters based on distance analysis
import importlib
import src.processing.distance_analysis as distance_analysis
importlib.reload(distance_analysis)
from src.processing.distance_analysis import (
    find_closest_excitatory_synapses,
    map_inhibitory_to_excitatory_clusters_by_distance,
    split_mixed_inhibitory_clusters_by_distance,
    compute_distances_within_clusters,
    analyze_e_i_relationships,
    compute_distance_statistics,
    define_cutoff_strategies,
    apply_distance_cutoff,
    print_distance_analysis_summary
)

# 1) Find closest excitatory synapses for each inhibitory synapse
print("Finding closest excitatory synapses for inhibitory synapses...")
syn_inh_df = find_closest_excitatory_synapses(syn_inh_df, syn_exec_df, geodesic_mat_full)

# 2) Map inhibitory synapses to excitatory clusters by distance
print("Mapping inhibitory synapses to excitatory clusters...")
valid_exec_ids = set(cluster_df["cluster_id"])
syn_inh_df = map_inhibitory_to_excitatory_clusters_by_distance(
    syn_inh_df, syn_exec_df, cluster_df
)

# 3) Split mixed inhibitory clusters
print("Splitting mixed inhibitory clusters...")
syn_inh_df = split_mixed_inhibitory_clusters_by_distance(syn_inh_df, valid_exec_ids)

# 4) Compute distances within clusters
print("Computing distances within clusters...")
syn_inh_df = compute_distances_within_clusters(syn_inh_df, syn_exec_df, geodesic_mat_full)



# 6) Compute distance statistics
print("Computing distance statistics...")
distance_stats = compute_distance_statistics(syn_inh_df)

# 7) Define cutoff strategies
cutoff_strategies = define_cutoff_strategies(distance_stats)

# 8) Apply distance cutoff (using dynamic cutoff based on overall density as in dc_initial_algo.py)
# Calculate dynamic cutoff: 1.00 / overall_density (following dc_initial_algo.py)
dynamical_cutoff = 1.00 / overall_density
print(f"Dynamic cutoff (1.00/overall_density): {dynamical_cutoff:.3f} μm")
print(f"Median cutoff: {cutoff_strategies['median']:.3f} μm")

# Use dynamic cutoff instead of median
cutoff_threshold = dynamical_cutoff
syn_inh_df_filtered = apply_distance_cutoff(syn_inh_df, cutoff_threshold)

# 9) Print comprehensive summary (without E/I relationship analysis)
print("=" * 80)
print("DISTANCE ANALYSIS SUMMARY")
print("=" * 80)

print(f"\n DISTANCE STATISTICS:")
print(f"   Total inhibitory synapses with valid distances: {distance_stats['total_synapses']}")
print(f"   Mean distance to closest E-synapse: {distance_stats['mean_distance']:.3f} μm")
print(f"   Median distance to closest E-synapse: {distance_stats['median_distance']:.3f} μm")
print(f"   Max distance to closest E-synapse: {distance_stats['max_distance']:.3f} μm")
print(f"   Min distance to closest E-synapse: {distance_stats['min_distance']:.3f} μm")
print(f"   Std distance to closest E-synapse: {distance_stats['std_distance']:.3f} μm")

print(f"\n FILTERING RESULTS:")
print(f"   Original inhibitory synapses: {len(syn_inh_df)}")
print(f"   Synapses with valid E-mapping: {len(syn_inh_df)}")
print(f"   Synapses after distance cutoff: {len(syn_inh_df_filtered)}")
print(f"   Filtering efficiency: {len(syn_inh_df_filtered)/len(syn_inh_df)*100:.1f}%")

print(f"\n CUTOFF STRATEGY:")
print(f"   Strategy: dynamic")
print(f"   Threshold: {cutoff_threshold:.3f} μm")
print("=" * 80)

print(" E/I distance-based mapping completed successfully")


Finding closest excitatory synapses for inhibitory synapses...
Mapping inhibitory synapses to excitatory clusters...
Splitting mixed inhibitory clusters...
Computing distances within clusters...
Computing distance statistics...
Dynamic cutoff (1.00/overall_density): 3.351 μm
Median cutoff: 1.184 μm
DISTANCE ANALYSIS SUMMARY

 DISTANCE STATISTICS:
   Total inhibitory synapses with valid distances: 169
   Mean distance to closest E-synapse: 1.969 μm
   Median distance to closest E-synapse: 1.184 μm
   Max distance to closest E-synapse: 15.639 μm
   Min distance to closest E-synapse: 0.000 μm
   Std distance to closest E-synapse: 2.707 μm

 FILTERING RESULTS:
   Original inhibitory synapses: 464
   Synapses with valid E-mapping: 464
   Synapses after distance cutoff: 146
   Filtering efficiency: 31.5%

 CUTOFF STRATEGY:
   Strategy: dynamic
   Threshold: 3.351 μm
 E/I distance-based mapping completed successfully


In [17]:
# =============================================================================
# 12. UPDATE SYNAPSE DATAFRAMES AFTER FILTERING
# =============================================================================
# Update syn_exec_df and syn_inh_df to reflect which clusters got filtered out

print("Updating synapse dataframes after cluster filtering...")

# 1) Update syn_exec_df to only include synapses in valid clusters
valid_cluster_ids = set(cluster_df['cluster_id'])
syn_exec_df_filtered = syn_exec_df[syn_exec_df['cluster_id'].isin(valid_cluster_ids)].copy()
print(f"E synapses: {len(syn_exec_df)} -> {len(syn_exec_df_filtered)} (filtered)")

# 2) syn_inh_df_filtered is already created in cell 11 (E/I distance-based mapping)
# No need to recreate it or merge with itself - it already contains all the necessary columns
print(f"I synapses: {len(syn_inh_df)} -> {len(syn_inh_df_filtered)} (filtered)")

# 3) Add I synapse cluster information to syn_exec_df_filtered
# This adds information about which I cluster each E synapse is closest to
print("Adding I synapse cluster information to E synapse dataframe...")

# Get the I synapse cluster information for each E cluster
e_cluster_i_cluster_info = syn_inh_df_filtered.groupby('cluster_id_exec').agg({
    'cluster_id_inh': 'first',  # Get the I cluster ID
    'min_dist_e_syn_in_clu': 'min'  # Get the minimum distance
}).reset_index()
e_cluster_i_cluster_info.columns = ['cluster_id', 'closest_i_cluster_id', 'min_dist_to_i_synapse']

# Merge this information with syn_exec_df_filtered
syn_exec_df_filtered = syn_exec_df_filtered.merge(
    e_cluster_i_cluster_info, 
    on='cluster_id', 
    how='left'
)

print(" Synapse dataframes updated after filtering")

Updating synapse dataframes after cluster filtering...
E synapses: 959 -> 557 (filtered)
I synapses: 464 -> 146 (filtered)
Adding I synapse cluster information to E synapse dataframe...
 Synapse dataframes updated after filtering


In [18]:
# =============================================================================
# 13. ENHANCED CLUSTER DATAFRAMES
# =============================================================================
# Create enhanced cluster dataframes with I synapse counts and additional properties

print("Creating enhanced cluster dataframes...")

# 1) Add I synapse counts to excitatory cluster_df
print("Adding I synapse counts to excitatory cluster dataframe...")

# Rename cluster_id to e_cluster_id for clarity
cluster_df = cluster_df.rename(columns={'cluster_id': 'e_cluster_id'})

# Count I synapses mapped to each E cluster
i_synapse_counts = syn_inh_df_filtered.groupby('cluster_id_exec').size().to_dict()
cluster_df['i_synapse_count'] = cluster_df['e_cluster_id'].map(i_synapse_counts).fillna(0).astype(int)

# Count I clusters mapped to each E cluster
i_cluster_counts = syn_inh_df_filtered.groupby('cluster_id_exec')['cluster_id_inh'].nunique().to_dict()
cluster_df['i_cluster_count'] = cluster_df['e_cluster_id'].map(i_cluster_counts).fillna(0).astype(int)

# 2) Create filtered inhibitory cluster dataframe
print("Creating filtered inhibitory cluster dataframe...")
syn_inh_cluster_df_filtered = syn_inh_df_filtered.groupby('cluster_id_inh').agg(
    i_synapse_count=('cluster_id_inh', 'size'),
    i_synapses=('id', lambda x: list(x)),
    mapped_e_cluster=('cluster_id_exec', 'first'),
    min_distance_to_e=('min_dist_e_syn_in_clu', 'min'),
    mean_distance_to_e=('min_dist_e_syn_in_clu', 'mean'),
    max_distance_to_e=('min_dist_e_syn_in_clu', 'max')
).reset_index()

# Rename cluster_id_inh to i_cluster_id for clarity
syn_inh_cluster_df_filtered = syn_inh_cluster_df_filtered.rename(columns={'cluster_id_inh': 'i_cluster_id'})

# 3) Create final I cluster assessment dataframe
print("Creating final I cluster assessment dataframe...")
final_i_cluster_df = syn_inh_cluster_df_filtered.copy()
final_i_cluster_df['e_cluster_id'] = final_i_cluster_df['mapped_e_cluster']
final_i_cluster_df = final_i_cluster_df.merge(
    cluster_df[['e_cluster_id', 'e_synapse_count', 'cluster_density', 'minimal_cable_length']], 
    left_on='e_cluster_id', 
    right_on='e_cluster_id', 
    how='left',
    suffixes=('', '_e')
)

# Add E/I ratio
final_i_cluster_df['e_i_ratio'] = final_i_cluster_df['e_synapse_count'] / final_i_cluster_df['i_synapse_count']
final_i_cluster_df['e_i_ratio'] = final_i_cluster_df['e_i_ratio'].replace([np.inf, -np.inf], np.nan)

# Remove redundant columns to avoid duplication
# Remove the old 'mapped_e_cluster' column since we now have 'e_cluster_id'
final_i_cluster_df = final_i_cluster_df.drop(columns=['mapped_e_cluster'])

print(" Enhanced cluster dataframes created successfully")

# Print summary statistics
print(f"\nEnhanced Cluster DataFrames Created:")
print(f"  Enhanced cluster_df shape: {cluster_df.shape}")
print(f"  syn_inh_cluster_df_filtered shape: {syn_inh_cluster_df_filtered.shape}")
print(f"  final_i_cluster_df shape: {final_i_cluster_df.shape}")

print(f"\nE Cluster Statistics:")
print(f"  E clusters with I synapses: {len(cluster_df[cluster_df['i_synapse_count'] > 0])}")
print(f"  Total I synapses mapped: {cluster_df['i_synapse_count'].sum()}")
print(f"  Total I clusters mapped: {cluster_df['i_cluster_count'].sum()}")

print(f"\nI Cluster Statistics:")
print(f"  I clusters after filtering: {len(syn_inh_cluster_df_filtered)}")
print(f"  I clusters with valid E mapping: {len(final_i_cluster_df[final_i_cluster_df['e_cluster_id'].notna()])}")
print(f"  Mean I cluster size: {syn_inh_cluster_df_filtered['i_synapse_count'].mean():.2f}")
print(f"  Mean distance to E synapses: {syn_inh_cluster_df_filtered['mean_distance_to_e'].mean():.2f} μm")

Creating enhanced cluster dataframes...
Adding I synapse counts to excitatory cluster dataframe...
Creating filtered inhibitory cluster dataframe...
Creating final I cluster assessment dataframe...
 Enhanced cluster dataframes created successfully

Enhanced Cluster DataFrames Created:
  Enhanced cluster_df shape: (206, 9)
  syn_inh_cluster_df_filtered shape: (121, 7)
  final_i_cluster_df shape: (121, 11)

E Cluster Statistics:
  E clusters with I synapses: 101
  Total I synapses mapped: 146
  Total I clusters mapped: 121

I Cluster Statistics:
  I clusters after filtering: 121
  I clusters with valid E mapping: 121
  Mean I cluster size: 1.21
  Mean distance to E synapses: 1.12 μm


In [19]:
# =============================================================================
# 14. DATAFRAME DESCRIPTIONS AND COMPACT SUMMARY
# =============================================================================
# Explain what each dataframe represents and provide compact summary statistics
import importlib
import src.processing.cluster_processing as cluster_processing
importlib.reload(cluster_processing)
from src.processing.cluster_processing import (
    compute_separated_cluster_statistics,
    print_separated_cluster_summary
)

print("=" * 80)
print("DATAFRAME DESCRIPTIONS")
print("=" * 80)

print(f"\n cluster_df:")
print(f"  Description: Excitatory clusters with their properties and I synapse counts")
print(f"  Shape: {cluster_df.shape}")
print(f"  Key columns: e_cluster_id, e_synapse_count, cluster_density, minimal_cable_length, i_synapse_count, i_cluster_count")

print(f"\n syn_inh_cluster_df_filtered:")
print(f"  Description: Inhibitory clusters after distance filtering with E cluster mapping")
print(f"  Shape: {syn_inh_cluster_df_filtered.shape}")
print(f"  Key columns: i_cluster_id, i_synapse_count, mapped_e_cluster, min/mean/max_distance_to_e")

print(f"\n final_i_cluster_df:")
print(f"  Description: Complete I cluster assessment with E cluster properties and E/I ratios")
print(f"  Shape: {final_i_cluster_df.shape}")
print(f"  Key columns: i_cluster_id, i_synapse_count, e_cluster_id, e_synapse_count, e_i_ratio, distance metrics")

print(f"\n syn_inh_df_filtered:")
print(f"  Description: Individual inhibitory synapses after distance filtering")
print(f"  Shape: {syn_inh_df_filtered.shape}")
print(f"  Key columns: id, cluster_id_inh, cluster_id_exec, min_dist_e_syn_in_clu")

# Compute separated cluster statistics
separated_stats = compute_separated_cluster_statistics(cluster_df)

# Print separated cluster summary
print_separated_cluster_summary(separated_stats)

# Overall E/I association summary
print(f"\n OVERALL E/I ASSOCIATION:")
print(f"  E clusters with I synapses: {len(cluster_df[cluster_df['i_synapse_count'] > 0])}")
print(f"  E clusters without I synapses: {len(cluster_df[cluster_df['i_synapse_count'] == 0])}")
print(f"  Total I synapses mapped: {cluster_df['i_synapse_count'].sum()}")
print(f"  Mean E/I ratio: {final_i_cluster_df['e_i_ratio'].mean():.2f}")

print("\n" + "=" * 80)
print(" SUMMARY COMPLETED")
print("=" * 80)


DATAFRAME DESCRIPTIONS

 cluster_df:
  Description: Excitatory clusters with their properties and I synapse counts
  Shape: (206, 9)
  Key columns: e_cluster_id, e_synapse_count, cluster_density, minimal_cable_length, i_synapse_count, i_cluster_count

 syn_inh_cluster_df_filtered:
  Description: Inhibitory clusters after distance filtering with E cluster mapping
  Shape: (121, 7)
  Key columns: i_cluster_id, i_synapse_count, mapped_e_cluster, min/mean/max_distance_to_e

 final_i_cluster_df:
  Description: Complete I cluster assessment with E cluster properties and E/I ratios
  Shape: (121, 11)
  Key columns: i_cluster_id, i_synapse_count, e_cluster_id, e_synapse_count, e_i_ratio, distance metrics

 syn_inh_df_filtered:
  Description: Individual inhibitory synapses after distance filtering
  Shape: (146, 19)
  Key columns: id, cluster_id_inh, cluster_id_exec, min_dist_e_syn_in_clu
COMPACT CLUSTER SUMMARY (SEPARATED BY I SYNAPSE ASSOCIATION)

 E CLUSTERS WITH I SYNAPSES (n=101):
   CLUST

In [20]:
# =============================================================================
# 15. COMPREHENSIVE FILTERING SUMMARY
# =============================================================================
# Print concise summary of filtering results (following dc_initial_algo.py style)

print("=" * 80)
print("COMPREHENSIVE FILTERING SUMMARY")
print("=" * 80)

# Synapse dataframe updates
print(f"\n SYNAPSE DATAFRAME UPDATES:")
print(f"  Updating synapse dataframes after cluster filtering...")
print(f"  E synapses: {len(syn_exec_df)} -> {len(syn_exec_df_filtered)} (filtered)")
print(f"  I synapses: {len(syn_inh_df)} -> {len(syn_inh_df_filtered)} (filtered)")
print(f"  Adding I synapse cluster information to E synapse dataframe...")
print(f"   Synapse dataframes updated after filtering")

print(f"\n FILTERED INHIBITORY SYNAPSE DATAFRAME:")
print(f"  Shape: {syn_inh_df_filtered.shape}")
print(f"  Columns: {list(syn_inh_df_filtered.columns)}")

# E synapse filtering summary
print(f"\n EXCITATORY SYNAPSES:")
print(f"  Initial E synapses: {len(syn_exec_df)}")
print(f"  E synapses after clustering and filtering: {len(syn_exec_df_filtered)}")  # All E synapses are clustered
print(f"  E synapses in valid clusters: {len(syn_exec_df)-len(syn_exec_df_filtered)}")  # Use filtered count

# E cluster filtering summary
print(f"\n EXCITATORY CLUSTERS:")
print(f"  Initial E clusters found: {len(unique_peaks)}")
print(f"  E clusters after density filtering: {len(cluster_df)}")
print(f"  E clusters filtered out: {len(unique_peaks) - len(cluster_df)}")

# I synapse filtering summary
print(f"\n INHIBITORY SYNAPSES:")
print(f"  Initial I synapses: {len(syn_inh_df)}")
print(f"  I synapses after distance filtering: {len(syn_inh_df_filtered)}")
print(f"  I synapses filtered out: {len(syn_inh_df) - len(syn_inh_df_filtered)}")

# I cluster filtering summary
print(f"\n INHIBITORY CLUSTERS:")
print(f"  Initial I clusters found: {len(clusters_dict_inh)}")
print(f"  I clusters after distance filtering: {len(syn_inh_cluster_df_filtered)}")
print(f"  I clusters filtered out: {len(clusters_dict_inh) - len(syn_inh_cluster_df_filtered)}")

# Cutoff information
print(f"\n CUTOFF INFORMATION:")
print(f"  Overall density: {overall_density:.5f} synapses/μm")
print(f"  Dynamic cutoff: {dynamical_cutoff:.3f} μm")
print(f"  Cable length: {neuron.cable_length:.2f} μm")

# E/I association summary
print(f"\n E/I ASSOCIATION:")
print(f"  E clusters with I synapses: {len(cluster_df[cluster_df['i_synapse_count'] > 0])}")
print(f"  Total I synapses mapped to E clusters: {cluster_df['i_synapse_count'].sum()}")
print(f"  Total I clusters mapped to E clusters: {cluster_df['i_cluster_count'].sum()}")

print("\n" + "=" * 80)
print(" COMPREHENSIVE FILTERING SUMMARY COMPLETED")
print("=" * 80)


COMPREHENSIVE FILTERING SUMMARY

 SYNAPSE DATAFRAME UPDATES:
  Updating synapse dataframes after cluster filtering...
  E synapses: 959 -> 557 (filtered)
  I synapses: 464 -> 146 (filtered)
  Adding I synapse cluster information to E synapse dataframe...
   Synapse dataframes updated after filtering

 FILTERED INHIBITORY SYNAPSE DATAFRAME:
  Shape: (146, 19)
  Columns: ['Unnamed: 0', 'id', 'Ipos3DX', 'Ipos3DY', 'Ipos3DZ', 'syn_size', 'pre_identity', 'pre_type', 'isOnSpine', 'SynapseVolume', 'closest_node_id', 'distance_to_node', 'cluster_id', 'closest_e_syn_id', 'min_dist_e_syn_tot', 'cluster_id_exec', 'cluster_id_inh_old', 'cluster_id_inh', 'min_dist_e_syn_in_clu']

 EXCITATORY SYNAPSES:
  Initial E synapses: 959
  E synapses after clustering and filtering: 557
  E synapses in valid clusters: 402

 EXCITATORY CLUSTERS:
  Initial E clusters found: 586
  E clusters after density filtering: 206
  E clusters filtered out: 380

 INHIBITORY SYNAPSES:
  Initial I synapses: 464
  I synapses a